# Data Extraction
This section details the process of extracting question data from a PDF file and corresponding answer keys from a TSV file.

## Importing libraries

* Pandas for data manipulation
* NumPy for the NaN datatype
* PyPDF for its ability to read the pfd files, although it requieres post-processing of the extracted text
* sqlite3 to create load the clean data into `bir_warehouse.db`

In [2]:
import pandas as pd
import numpy as np
from pypdf import PdfReader
import sqlite3

## Selecting year and type of exam

This function, `compute_expected_rows`, takes the exam year and topic as input and returns metadata crucial for subsequent data cleaning and loading steps. This metadata includes the expected maximum number of rows in the final DataFrame, the number of questions per exam, and the acronym for the exam type.  The maximum number of rows is calculated based on the number of questions and options per question for each year, following a specific pattern:

*   2024-2021: 210 questions, 4 options each
*   2020-2019: 185 questions, 4 options each
*   2018-2015: 235 questions, 4 options each
*   2014-2012: 235 questions, 5 options each
*   2011-2004: 260 questions, 5 options each

This information is essential for validating data integrity during the ETL process.  For example, the expected row count helps verify that all questions and options have been correctly extracted and transformed. The exam type acronym aids in consistent file naming and data organization.

In [3]:
def compute_expected_rows(year: int, topic: str) -> tuple:
    """Computes and returns metadata about the exam for a given year and topic

    Args:
        year (int): The year of the exam
        topic (str): The topic of the exam

    Returns:
        tuple: Contains:
            - max_rows: The expected maximum number of rows in the DataFrame
            - num_options: The number of questions in the exam
            - exam_type_acronym: The acronym for the exam type

    """
    info_year_dict: dict = {
        2011: [260, 5],
        2014: [235, 5],
        2018: [235, 4],
        2020: [185, 4],
        2021: [210, 4],
    }
    save_name_dict: dict = {
        "BIOLOGÍA": "bir",
        "FARMACIA": "fir",
        "QUÍMICA": "qir",
        "MEDICINA": "mir",
    }
    max_rows: int = 0
    num_options: int = 0
    for target_year, info_list in info_year_dict.items():
        if year <= target_year:
            max_rows = (info_list[0] * info_list[1]) + info_list[0]
            num_options = info_list[1]
            break
    else:
        max_rows = 1050
        num_options = 4
    return max_rows, num_options, save_name_dict[topic]

### Add year of the exam and topic 

In [4]:
year: int = 2022
topic: str = "FARMACIA" # Select from: BIOLOGÍA, FARMACIA, QUÍMICA, MEDICINA
total_num_rows: int
num_questions: int
exam_acronym: str
total_num_rows, num_questions, exam_acronym = compute_expected_rows(year, topic)
save_format: str = f"../data/clean/clean_{exam_acronym}_{year}.csv"

In [4]:
add_list: list = [x for x in range(19, 79, 1)] + [85, 86, 87] + [x for x in range(123, 151, 1)] + [x for x in range(183, 189, 1)] + [x for x in range(195, 200, 1)] + [202, 204, 205, 206, 208, 210]

## Extracting Questions from Pdf file

This section focuses on extracting the textual content of the exam questions from the provided PDF file.

**Path Definition:**

The path to the PDF file is dynamically constructed using the provided `year` and `topic` variables. This ensures flexibility and allows the script to process different exam files without manual path adjustments.

Finally it stores the output in a DataFrame and the first 10 rows of the resulting DataFrame are displayed for a quick preview of the loaded data.

In [5]:
path: str = f"../data/raw/type_1/Raw_Cuaderno_{year}_{topic}_0_C.pdf"
with PdfReader(path) as pdf_file:
    full_text: list = []
    for n in range(2, len(pdf_file.pages)):
        page = pdf_file.pages[n]
        text: str = page.extract_text()
        full_text.extend(text.splitlines())
raw_exam_df: pd.DataFrame = pd.DataFrame(full_text, columns=["text"])
raw_exam_df.head(10)

,text
0,
1,- 1 -
2,1. Saquinavir y ritonavir son fármacos utiliza...
3,en el tratamiento del SIDA que son además un
4,ejemplo de diseño racional basado en la estruc-
5,tura. ¿Cuál de las siguientes afirmaciones des-
6,cribe correctamente a estos compuestos?:
7,
8,1. Son peptidomiméticos diseñados para inhibir
9,la proteasa del VIH y actuar como análogos


## Extracting Answers from .tsv file into a DataFrame

This section focuses on extracting the answer key data from the provided .tsv file and loading it into a Pandas DataFrame.

**Path Definition and Data Loading:**

The path to the .tsv file is dynamically constructed using the `year` and `topic` variables, mirroring the approach used for the PDF file. This ensures consistency and flexibility in processing different exam files. 

The `pd.read_table()` function is then used to read the .tsv file directly into a Pandas DataFrame. Finally, the first 10 rows of the resulting DataFrame are displayed for a quick preview of the loaded data.

In [6]:
raw_answers_df: pd.DataFrame = pd.read_table(f"../data/raw/type_1/Raw_Cuaderno_{year}_{topic}_0_C_Respuestas.tsv")
raw_answers_df.head(10)

,V,RC,V.1,RC.1,V.2,RC.2,V.3,RC.3,V.4,RC.4
0,1,1,43,4,85,2,127,2,169,2
1,2,3,44,1,86,2,128,1,170,3
2,3,2,45,4,87,2,129,1,171,4
3,4,1,46,2,88,1,130,3,172,2
4,5,2,47,1,89,4,131,2,173,1
5,6,4,48,4,90,3,132,2,174,3
6,7,2,49,2,91,1,133,3,175,3
7,8,3,50,2,92,1,134,4,176,1
8,9,3,51,1,93,2,135,4,177,4
9,10,2,52,2,94,3,136,2,178,3


# Data Transformation

## Exam pdf

### Removing white space

This section focuses on cleaning the raw text extracted from the PDF exam file, preparing it for further processing. This involves removing extraneous whitespace, handling empty lines, and resetting the DataFrame index.

**1. Removing Page Number Artifacts:**

The extracted text may contain page number artifacts, which are removed by filtering out rows containing the string "Página".

In [7]:
raw_exam_df = raw_exam_df[~raw_exam_df["text"].str.contains("Página")]
raw_exam_df = raw_exam_df[~raw_exam_df["text"].str.contains("Pagina")]
raw_exam_df = raw_exam_df[~raw_exam_df["text"].str.contains(topic)]

footer_list: list = [f"- {npag} -" for npag in range(0,30,1)]

for footer in footer_list:
    raw_exam_df["text"] = raw_exam_df["text"].str.replace(footer, "")

**2. Striping Leading/Trailing Whitespaces**

Leading and trailing whitespace characters are removed from each text entry using the .str.strip() method.

In [8]:
raw_exam_df = raw_exam_df["text"].str.strip()

**3. Handling Empty Lines**

Empty lines, represented as empty strings, are replaced with NaN (Not a Number) values. 

Then rows containing NaN values, representing empty lines, are removed from the DataFrame using .dropna()

In [9]:
raw_exam_df = raw_exam_df.replace("", np.nan)
raw_exam_df = raw_exam_df.dropna()

**4. Reseting DataFrame Index**

The DataFrame index is reset after removing rows, ensuring a contiguous index and dropping the old index.

In [10]:
raw_exam_df = raw_exam_df.reset_index(drop=True)

**5. Displays the first 10 rows of the DataFrame**

In [11]:
raw_exam_df.head(10)

0    1. Saquinavir y ritonavir son fármacos utilizados
1         en el tratamiento del SIDA que son además un
2      ejemplo de diseño racional basado en la estruc-
3      tura. ¿Cuál de las siguientes afirmaciones des-
4             cribe correctamente a estos compuestos?:
5       1. Son peptidomiméticos diseñados para inhibir
6           la proteasa del VIH y actuar como análogos
7                            del estado de transición.
8     2. Son análogos de los ácidos nucleicos y actúan
9           como inhibidores competitivos de la trans-
Name: text, dtype: object

In [12]:
raw_exam_df.tail(10)

2197                                      2. Es una odds.
2198                                      3. Es una tasa.
2199                                     4. Es una razón.
2200    210. En un método analítico, la concordancia e...
2201     el valor medido y el valor real se relaciona con
2202                    la siguiente propiedad analítica:
2203                                  1. La selectividad.
2204                                     2. La precisión.
2205                                     3. La exactitud.
2206                                  4. La sensibilidad.
Name: text, dtype: object

### Joining Truncated Lines

This section addresses the issue of truncated lines and multi-line questions or options within the extracted exam text. A custom function, `process_multi_line_str`, is used to concatenate these lines, ensuring that each question and option is presented as a single, coherent string.

**Function: `process_multi_line_str(df)`**

This function iterates through the DataFrame, performing two main tasks:

1.  **Joining Truncated Words:**
    * It checks if a line ends with a hyphen ("-"), indicating a word that has been truncated at the end of a line.
    * If a line is truncated, it removes the hyphen and appends the content of the following line to the current line.
    * The following line is then removed from the DataFrame, and the index is reset.

2.  **Joining Multi-Line Questions/Options:**
    * It checks if the first character of the next line can be converted to an integer. This is used as a heuristic to identify the start of a new question or option.
    * If the next line does not start with an integer (i.e., it's a continuation of the current question or option), it is appended to the current line, separated by a space.
    * The next line is then removed, and the index is reset.

In [13]:
def process_multi_line_str(df: pd.DataFrame) -> pd.DataFrame:
    """Function to handle truncated lines

    Args:
        df (pd.DataFrame): A pandas dataframe

    Returns:
        pd.DataFrame: Returns the DataFrame without truncated lines
    """
    i: int = 0
    while i < len(df) - 1:
        if i < len(df) -1  and df.iloc[i].endswith("-"):
            df.iloc[i] = df.iloc[i][:-1] + df.iloc[i + 1]
            df = df.drop(i + 1)
            df = df.reset_index(drop=True)
        else:
            i += 1
    n: int = 0
    while n < len(df) -1:
        if n + 1 < len(df):
            try:
                int(df.iloc[n + 1][0:1])
                n += 1
            except ValueError:
                df.iloc[n] = df.iloc[n] + " " + df.iloc[n + 1]
                df = df.drop(n +1)
                df = df.reset_index(drop= True)
        else:
            break
    return df

The process_multi_line_str function is applied to the cleaned raw_exam_df DataFrame, and the resulting DataFrame is stored in exam_df_concat. The first few rows of the concatenated DataFrame are displayed to verify the results.

In [14]:
exam_df_concat: pd.DataFrame = process_multi_line_str(raw_exam_df)
exam_df_concat.head(10)

0    1. Saquinavir y ritonavir son fármacos utiliza...
1    1. Son peptidomiméticos diseñados para inhibir...
2    2. Son análogos de los ácidos nucleicos y actú...
3    3. Son peptidomiméticos diseñados para inhibir...
4    4. Son compuestos que contienen en su estructu...
5    2. Las penicilinas poseen una estructura bicíc...
6    1. Confiere una elevada tensión que dificulta ...
7    2. Mimetiza el enlace glicosídico entre Naceti...
8    3. Mimetiza el dímero de D-Ala-D-Ala presente ...
9    4. Puede ser modificado bioisostéricamente, su...
Name: text, dtype: object

### Validating and Correction Incorrect Rows

This section focuses on validating the number of rows in the cleaned DataFrame against the expected number and correcting any rows that do not conform to the expected format.

**1. Calculating Expected and Actual Incorrect Rows:**

We calculate the expected number of incorrect rows by subtracting the total expected rows from the current number of rows in the DataFrame. We then identify the actual number of incorrect rows by counting the rows that do not end with a period ("."), a colon (":") or interrogation sign ("?").

In [15]:
num_rows_incorrect_expected: int = len(exam_df_concat) - total_num_rows
num_rows_incorrect: int = exam_df_concat[~exam_df_concat.str.endswith((".", ":", "?"))].count()
id_rows_incorrect: list = exam_df_concat[~exam_df_concat.str.endswith((".", ":", "?"))].index.to_list()
print(f"Number of expected incorrect rows = {num_rows_incorrect_expected}")
print(f"Number of rows not ending with '.', ':' or '?' = {num_rows_incorrect}")
print(id_rows_incorrect)

Number of expected incorrect rows = 6
Number of rows not ending with '.', ':' or '?' = 7
[345, 352, 475, 504, 521, 701, 1015]


**2. Function: print_rows_incorrrect(wrong_id, df):**

This function takes a list of incorrect row indices and the DataFrame as input. It then prints the row identified as incorrect along with the 2 rows before and 2 rows after, providing context to help determine how to fix the error.

In [16]:
def print_rows_incorrrect(wrong_id: list[int], df: pd.DataFrame) -> None:
    """Print +/- 2 rows for context

    Args:
        wrong_id (list): Generated list of wrong row_id
        df (pd.DataFrame): DataFrame corresponding with the worng_id list
    """
    for wid in wrong_id:
        print(f"Id to fix {wid}:")
        print(df.iloc[wid -2: wid +3])

**3. Function: fix_incorrect(num_row, df):**

This function takes a list of row indices to fix and the DataFrame. It iterates through the indices in reverse order to avoid index shifting problems. Each incorrect row is concatenated with the row immediately following it, and the subsequent row is then dropped. The DataFrame index is reset after each concatenation.

In [17]:
def fix_incorrect(num_row: list[int], df: pd.DataFrame) -> pd.DataFrame:
    """Fix the incorrect rows in the DataFrame

    Args:
        num_row (list): List of ids to fix
        df (pd.DataFrame): The DataFrame to fix

    Returns:
        pd.DataFrame: Fixed
    """
    num_row: list = sorted(num_row, reverse=True)
    for n in num_row:
        df.iloc[n] = df.iloc[n] + df.iloc[n + 1]
        df = df.drop(n + 1)
        df = df.reset_index(drop= True)
    return df

**4. Validation and Correction Logic:**

We compare the expected and actual number of incorrect rows. If they match, we proceed to fix the incorrect rows using the `fix_incorrect` function. Otherwise, we print the context of the incorrect rows and raise a warning, indicating that manual inspection and correction are required.

In [18]:
if num_rows_incorrect_expected == num_rows_incorrect:
    exam_df_fixed = fix_incorrect(id_rows_incorrect, exam_df_concat)
    df_correct_rows: bool = len(exam_df_fixed) == num_rows_incorrect_expected
else:
    print_rows_incorrrect(id_rows_incorrect, exam_df_concat)
    # raise Warning("More incorrect rows that expected, uncomment the lines below and add ids to the list, and comment this line")
    rows_fix: list[int] = [345, 352, 475, 504, 701, 1015]
    exam_df_fixed: pd.DataFrame = fix_incorrect(rows_fix, exam_df_concat)

Id to fix 345:
343    3. La noradrenalina se utiliza frecuentemente ...
344    4. En un shock cardiogénico no hay manifestaci...
345    70. AG es una mujer de 82 años, con antecedent...
346    24 horas, diabetes mellitus tipo 2 insulinodep...
347    1. Realizar un control de ritmo con fármacos q...
Name: text, dtype: object
Id to fix 352:
350    4. Es esencial observar el tamaño de las ondas...
351    71. Acerca del accidente isquémico transitorio...
352      1. Se trata de una lesión isquémica de menos de
353    24 horas de evolución, detectada radiológicame...
354    2. Cuando se diagnostica, es necesario comenza...
Name: text, dtype: object
Id to fix 475:
473    1. La afinidad de la insulina detemir es 4-5 v...
474    2. La insulina glargina se formula a un pH bás...
475    3. En la insulina degludec, los monómeros se l...
476                                                42 h.
477    4. En la insulina NPH la dosis regula el perfi...
Name: text, dtype: object
Id to fix 504:
502    

**5. Verification of Corrected Rows:**

We print the expected total number of rows and the actual number of rows in the corrected DataFrame to verify the results of the correction process.

In [19]:
print(f"Correct number of rows = {total_num_rows}")
print(exam_df_fixed.shape[0])

Correct number of rows = 1050
1050


### Pivoting the DataFrame

This section focuses on transforming the cleaned exam data into a structured format suitable for analysis. The DataFrame is pivoted to create columns for each question and its corresponding options.

**1. Preparing the DataFrame for Pivoting:**

The `exam_df_fixed` Series is converted to a DataFrame, and a 'group' column is created to identify each question and its options. An 'option_num' column is then generated to enumerate the options within each group.


In [20]:
exam_df_fixed = exam_df_fixed.to_frame()
groups: pd.Series = pd.Series((exam_df_fixed.index // (num_questions+1)) +1)
exam_df_fixed["group"] = groups  
exam_df_fixed["option_num"] = exam_df_fixed.groupby("group").cumcount() + 1

**2. Pivoting the DataFrame:**

The DataFrame is pivoted using the 'group' column as the index, the 'option_num' column as the columns, and the 'text' column as the values. The index is then reset.

In [21]:
exam_df_pivot: pd.DataFrame = exam_df_fixed.pivot(index="group", columns="option_num", values="text")
exam_df_pivot = exam_df_pivot.reset_index()

**3. Renaming Columns:**

The columns are renamed to more descriptive names, such as "Question", "Option_1", "Option_2", etc.

In [22]:
key_list: list = [x for x in range(1, num_questions+2, 1)]
val_list: list = ["Question"] + [f"Option_{x}" for x in range(1, 4 + 1, 1)]

exam_df_pivot = exam_df_pivot.rename_axis(None, axis=1).rename(columns=dict(zip(key_list, val_list)))

**4. Dropping the 'group' Column:**

The 'group' column is dropped as it is no longer needed

In [23]:
exam_df_pivot = exam_df_pivot.drop(columns=["group"])

**5. Displaying the Pivoted DataFrame:**

The first 10 rows of the pivoted DataFrame are displayed to verify the results of the transformation.

In [24]:
exam_df_pivot.head(10)

,Question,Option_1,Option_2,Option_3,Option_4
0,1. Saquinavir y ritonavir son fármacos utiliza...,1. Son peptidomiméticos diseñados para inhibir...,2. Son análogos de los ácidos nucleicos y actú...,3. Son peptidomiméticos diseñados para inhibir...,4. Son compuestos que contienen en su estructu...
1,2. Las penicilinas poseen una estructura bicíc...,1. Confiere una elevada tensión que dificulta ...,2. Mimetiza el enlace glicosídico entre Naceti...,3. Mimetiza el dímero de D-Ala-D-Ala presente ...,"4. Puede ser modificado bioisostéricamente, su..."
2,3. Los β-bloqueantes fueron diseñados para act...,"1. Introduciendo un grupo pequeño (H, metilo) ...",2. Introduciendo un grupo moderadamente volumi...,3. Eliminando el grupo catecol y sustituyéndol...,4. Epimerizando el grupo hidroxilo.
3,4. El sistema de quinolina forma parte de la e...,1. Anilina con ácido sulfúrico y glicerina en ...,2. Piridina con malonato de dietilo en medio b...,3. Fenilpropilamina en condiciones de ciclación.,4. Anilina con propanol en condiciones de sust...
4,5. ¿Cuál de los siguientes antagonistas de los...,1. Ketamina.,2. Amantadina.,3. Ganciclovir.,4. Dextrometorfano.
5,6. ¿Con qué nombre se describen los fármacos q...,1. Morfinanos.,2. Fenilpiperidinas.,3. Anilinopiperidinas.,4. Benzomorfanos.
6,"7. En el diseño de las estatinas, ¿qué fragmen...",1. El fragmento de decalina.,2. El fragmento de β-hidroxi-δ-lactona o su co...,3. El anillo aromático halogenado.,4. El grupo isopropilo.
7,8. ¿Cuál de las siguientes ecuaciones define a...,1. P = [fármaco]agua /[fármaco]octanol.,2. P = Log[fármaco]agua.,3. P = [fármaco]octanol /[fármaco]agua.,4. P = Log[fármaco]octanol.
8,9. ¿Qué tipo de intermedio reactivo se forma e...,1. Quinona.,"2. 1,2-Diol.",3. Epóxido.,4. Radical hidroxilo.
9,"10. La metodología utilizada para preparar, en...",1. Química supramolecular.,2. Química combinatoria.,3. Química divergente.,4. Química sostenible.


## Answers tsv

This section focuses on cleaning and consolidating the answer key data from the TSV file. The original data contains duplicate columns, which are processed and combined into a single, clean DataFrame.

**1. Defining Column Lists:**

We define lists containing the original column names and the duplicate column names.

In [25]:
original_col: list[str] = ["V", "RC"]
duplicate_cols: list[str] = ["V.1", "RC.1", "V.2", "RC.2", "V.3", "RC.3", "V.4", "RC.4"]

**2. Processing Duplicate Columns:**

We iterate through the duplicate_cols list in pairs, extracting the corresponding columns, renaming them to the original column names, and appending them to a list of DataFrames.

In [26]:
answers_df_list: list[pd.DataFrame] = [raw_answers_df[original_col]]

for i in range(0, len(duplicate_cols), 2):
    pair_cols: list = duplicate_cols[i:i+2]
    df_pair_col: pd.DataFrame = raw_answers_df[pair_cols].rename(columns={pair_cols[0]: "V", pair_cols[1]: "RC"})
    answers_df_list.append(df_pair_col)

**3. Concatenating DataFrames:**

The list of DataFrames is concatenated into a single DataFrame, ignoring the original index.

In [27]:
answers_df_clean: pd.DataFrame = pd.concat(answers_df_list, ignore_index=True)

**4. Displaying the Cleaned DataFrame:**

The cleaned DataFrame is printed to verify the results of the consolidation process.

In [28]:
answers_df_clean.head(10)

,V,RC
0,1,1
1,2,3
2,3,2
3,4,1
4,5,2
5,6,4
6,7,2
7,8,3
8,9,3
9,10,2


## Joining the DataFrames

This section focuses on merging the processed exam questions DataFrame with the cleaned answers DataFrame, performing final data cleaning, and verifying the integrity of the joined data.

**1. Joining the DataFrames:**

The pivoted exam questions DataFrame (`exam_df_pivot`) and the cleaned answers DataFrame (`answers_df_clean`) are joined horizontally using `pd.concat`. The 'V0' column, which represents the question number from the answers DataFrame, is then dropped. The first 10 rows of the joined DataFrame are displayed.

In [29]:
processed_exam_df: pd.DataFrame = pd.concat([exam_df_pivot, answers_df_clean], axis=1)
processed_exam_df = processed_exam_df.drop(columns=["V"])
processed_exam_df.head(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC
0,1. Saquinavir y ritonavir son fármacos utiliza...,1. Son peptidomiméticos diseñados para inhibir...,2. Son análogos de los ácidos nucleicos y actú...,3. Son peptidomiméticos diseñados para inhibir...,4. Son compuestos que contienen en su estructu...,1
1,2. Las penicilinas poseen una estructura bicíc...,1. Confiere una elevada tensión que dificulta ...,2. Mimetiza el enlace glicosídico entre Naceti...,3. Mimetiza el dímero de D-Ala-D-Ala presente ...,"4. Puede ser modificado bioisostéricamente, su...",3
2,3. Los β-bloqueantes fueron diseñados para act...,"1. Introduciendo un grupo pequeño (H, metilo) ...",2. Introduciendo un grupo moderadamente volumi...,3. Eliminando el grupo catecol y sustituyéndol...,4. Epimerizando el grupo hidroxilo.,2
3,4. El sistema de quinolina forma parte de la e...,1. Anilina con ácido sulfúrico y glicerina en ...,2. Piridina con malonato de dietilo en medio b...,3. Fenilpropilamina en condiciones de ciclación.,4. Anilina con propanol en condiciones de sust...,1
4,5. ¿Cuál de los siguientes antagonistas de los...,1. Ketamina.,2. Amantadina.,3. Ganciclovir.,4. Dextrometorfano.,2
5,6. ¿Con qué nombre se describen los fármacos q...,1. Morfinanos.,2. Fenilpiperidinas.,3. Anilinopiperidinas.,4. Benzomorfanos.,4
6,"7. En el diseño de las estatinas, ¿qué fragmen...",1. El fragmento de decalina.,2. El fragmento de β-hidroxi-δ-lactona o su co...,3. El anillo aromático halogenado.,4. El grupo isopropilo.,2
7,8. ¿Cuál de las siguientes ecuaciones define a...,1. P = [fármaco]agua /[fármaco]octanol.,2. P = Log[fármaco]agua.,3. P = [fármaco]octanol /[fármaco]agua.,4. P = Log[fármaco]octanol.,3
8,9. ¿Qué tipo de intermedio reactivo se forma e...,1. Quinona.,"2. 1,2-Diol.",3. Epóxido.,4. Radical hidroxilo.,3
9,"10. La metodología utilizada para preparar, en...",1. Química supramolecular.,2. Química combinatoria.,3. Química divergente.,4. Química sostenible.,2


**2. Identifying and Handling Missing Values:**

Missing values are then replaced with 0.

In [30]:
processed_exam_df = processed_exam_df.fillna(0)

**3. Verifying Missing Values and Data Types:**

The number of missing values in each column is checked, and the data types of the columns are displayed. The 'RC' (correct answer) column is then converted to integer type.

In [31]:
processed_exam_df.isnull().sum()
processed_exam_df["RC"] = processed_exam_df["RC"].astype(int)
processed_exam_df.dtypes

Question    object
Option_1    object
Option_2    object
Option_3    object
Option_4    object
RC           int32
dtype: object

**4. Adding year column**

A column containing the year of the exam is added to the DataFrame.

In [32]:
processed_exam_df["year"] = year

**5. Displaying the Final DataFrame:**

The first 10 rows of the final joined and cleaned DataFrame are displayed.

In [33]:
processed_exam_df.head(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
0,1. Saquinavir y ritonavir son fármacos utiliza...,1. Son peptidomiméticos diseñados para inhibir...,2. Son análogos de los ácidos nucleicos y actú...,3. Son peptidomiméticos diseñados para inhibir...,4. Son compuestos que contienen en su estructu...,1,2022
1,2. Las penicilinas poseen una estructura bicíc...,1. Confiere una elevada tensión que dificulta ...,2. Mimetiza el enlace glicosídico entre Naceti...,3. Mimetiza el dímero de D-Ala-D-Ala presente ...,"4. Puede ser modificado bioisostéricamente, su...",3,2022
2,3. Los β-bloqueantes fueron diseñados para act...,"1. Introduciendo un grupo pequeño (H, metilo) ...",2. Introduciendo un grupo moderadamente volumi...,3. Eliminando el grupo catecol y sustituyéndol...,4. Epimerizando el grupo hidroxilo.,2,2022
3,4. El sistema de quinolina forma parte de la e...,1. Anilina con ácido sulfúrico y glicerina en ...,2. Piridina con malonato de dietilo en medio b...,3. Fenilpropilamina en condiciones de ciclación.,4. Anilina con propanol en condiciones de sust...,1,2022
4,5. ¿Cuál de los siguientes antagonistas de los...,1. Ketamina.,2. Amantadina.,3. Ganciclovir.,4. Dextrometorfano.,2,2022
5,6. ¿Con qué nombre se describen los fármacos q...,1. Morfinanos.,2. Fenilpiperidinas.,3. Anilinopiperidinas.,4. Benzomorfanos.,4,2022
6,"7. En el diseño de las estatinas, ¿qué fragmen...",1. El fragmento de decalina.,2. El fragmento de β-hidroxi-δ-lactona o su co...,3. El anillo aromático halogenado.,4. El grupo isopropilo.,2,2022
7,8. ¿Cuál de las siguientes ecuaciones define a...,1. P = [fármaco]agua /[fármaco]octanol.,2. P = Log[fármaco]agua.,3. P = [fármaco]octanol /[fármaco]agua.,4. P = Log[fármaco]octanol.,3,2022
8,9. ¿Qué tipo de intermedio reactivo se forma e...,1. Quinona.,"2. 1,2-Diol.",3. Epóxido.,4. Radical hidroxilo.,3,2022
9,"10. La metodología utilizada para preparar, en...",1. Química supramolecular.,2. Química combinatoria.,3. Química divergente.,4. Química sostenible.,2,2022


### Filtering by relevant questions

**1. Filtering for id**

A list of the number of question is used to filter the DataFrame, but first doing a -1 so it matches with the id. 

Finally the indexes are reseted

In [34]:
id_filter = [x-1 for x in add_list]
processed_exam_df_filtered = processed_exam_df.iloc[id_filter]
processed_exam_df_filtered = processed_exam_df_filtered.reset_index(drop= True)
print(f"Expected number of rows = {len(id_filter)}\nNumber of rows in filtered df = {processed_exam_df_filtered.shape[0]}")

Expected number of rows = 108
Number of rows in filtered df = 108


**2. Showing the filtered DataFrame**

Using head and tail

In [35]:
processed_exam_df_filtered.head(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
0,19. ¿Cuál es la respuesta correcta respecto al...,1. Es una enterobacteria.,2. Es un bacilo gramnegativo no fermentador.,3. Es una espiroqueta.,4. Es una bacteria intracelular.,1,2022
1,20. ¿Cuál es la respuesta INCORRECTA respecto ...,1. Pertenece a la familia Paramyxoviridae.,2. El único hospedador es el ser humano.,3. Puede producir epidídimo-orquitis.,4. Es un virus ARN monocatenario con simetría ...,4,2022
2,21. ¿Cuál de las siguientes respuestas es corr...,1. Es un virus ARN bicatenario de simetría hel...,2. Reconoce el receptor humano ACE2 (enzima co...,3. El gen estructural S es idéntico al del vir...,4. Pertenece a la familia Orthomyxoviridae.,2,2022
3,22. ¿Por qué es difícil el aislamiento microbi...,1. Solo crece en anaerobiosis tras una incubac...,2. Necesita medios de cultivo enriquecidos con...,3. Necesita medios de cultivo enriquecidos con...,4. Necesita sistemas de cultivo celulares y un...,2,2022
4,23. ¿Cuál de las siguientes afirmaciones es IN...,1. Son pequeños cocobacilos o bacilos gramnega...,2. Bartonella bacilliformis es responsable de ...,3. Bartonella quintana se transmite de persona...,4. Bartonella henselae s e t r a n s m i t e ...,4,2022
5,24. Señale cuál de las siguientes afirmaciones...,1. Es un bacilo gramnegativo curvo.,2. Gran parte de las infecciones que produce e...,3. Para su aislamiento a partir de heces se re...,4. Es resistente a los macrólidos.,4,2022
6,25. ¿Cuál de los siguientes microorganismos es...,1. Listeria monocytogenes.,2. Bordetella pertussis.,3. Francisella tularensis.,4. Helicobacter pylori.,1,2022
7,26. Señale la respuesta correcta sobre el géne...,1. Mycobacterium chelonae es una micobacteria ...,2. Mycobacterium kansasii es una micobacteria ...,3. Mycobacterium tuberculosis da positiva la ...,4. Mycobacterium bovis es sensible a pirazina...,3,2022
8,27. Las micosis sistémica s endémicas son aque...,1. Dermatofitos.,2. Hongos dimórficos.,3. Mucorales.,4. Hongos hialinos.,2,2022
9,28. Señale cuál de los siguientes microorganis...,1. Enterococcus casseliflavus.,2. Abiotrophia defectiva.,3. Streptococcus gallolyticus.,4. Gemella haemolysans.,1,2022


In [36]:
processed_exam_df_filtered.tail(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
98,196. La reacción en el centro germinal es una ...,"1. La expansión clonal, la proliferación, el c...","2. La hipermutación somática, la edición del T...","3. La expansión clonal, el cambio de isotipo, ...","4. La hipermutación somática, el reordenamient...",3,2022
99,197. La activación de las células B para produ...,1. Como contacto directo con una célula T coop...,2. Como contacto directo con una célula T coop...,3. Como contacto directo con una célula T coop...,4. Como contacto directo con una célula T coop...,3,2022
100,198. ¿Cuál de las siguientes moléculas partici...,1. CD3.,2. CD4 y CD8.,3. CD5.,4. CD28.,4,2022
101,199. Indique cuál de las siguientes afirmacion...,1. Las IDs que afectan a los mecanismos de fag...,2. Las IDs de proteínas del sistema del comple...,3. La severidad de las IDs causadas por linfoc...,4. Las IDs primarias son congénitas y las secu...,2,2022
102,202. ¿Cuál de los siguientes microorganismos s...,1. Streptobacillus moniliformis.,2. Bartonella bacilliformis.,3. Rickettsia typhi.,4. Coxiella burnetti.,1,2022
103,204. ¿Cuál de los siguientes antivíricos está ...,1. Foscarnet.,2. Letermovir.,3. Cidofovir.,4. Valganciclovir.,2,2022
104,205. Señale la opción correc ta en relación a ...,1. El factor activador de plaquetas pertenece ...,2. Son lípidos insaponificables si tienen carg...,"3. El fosfatidilinositol, que se encuentra lib...",4. La cardiolipina es el principal componente ...,1,2022
105,206. ¿Cuál de los siguientes antibióticos es i...,1. Ceftriaxona.,2. Levofloxacino.,3. Linezolid.,4. Daptomicina.,4,2022
106,208. Los llamados inhibidores mixtos son un ti...,1. Se denominan así en referencia a que pueden...,"2. Afectan a la Km aparente, pero no a la velo...","3. En la práctica, sólo se observan en enzimas...",4. Pueden provocar un aumento o disminución de...,4,2022
107,"210. En un método analítico, la concordancia e...",1. La selectividad.,2. La precisión.,3. La exactitud.,4. La sensibilidad.,3,2022


# Data Loading

## Saving in csv file

The `processed_exam_df_filtered` DataFrame, containing the cleaned and transformed exam data, is saved to a CSV file with the name generatd in the first step in the "data" directory. The `index=False` argument ensures that the DataFrame index is not written to the file, resulting in a cleaner output. This CSV file can be used for further analysis or as input for other applications.

In [37]:
processed_exam_df_filtered.to_csv(save_format, index= False)

## Writing into data warehouse

This section details the process of loading the cleaned and transformed exam data into an SQLite database, serving as a data warehouse for further analysis and querying.

**1. Database Connection and Path:**

The path to the SQLite database file is defined, and a connection is established using the `sqlite3` library.

In [5]:
db_path: str = "../data/clean/bir_warehouse.db"

**2. SQL Queries Definition:**

Two SQL queries are defined: one for inserting questions into the questions table and another for inserting options into the questions_options table.

In [6]:
query_question: str = """
INSERT INTO questions (exam_year, exam_subject, question) 
VALUES((SELECT id_year FROM year WHERE year_name = ?),
    (SELECT id_type FROM exam WHERE exam_type = ?),
    ?);
"""

query_options: str = """
INSERT INTO questions_options (question_id, option_num, option_text, is_correct)
VALUES((SELECT q.id 
        FROM questions AS q
        JOIN year AS y ON q.exam_year = y.id_year
        WHERE q.question = ? AND y.year_name = ?),
    ?,
    ?,
    ?)
"""

**3. Database Interaction and Data Insertion:**

A with statement is used to establish a connection to the SQLite database, ensuring that the connection is properly closed after use. A cursor is created, and the processed_exam_df DataFrame is iterated over to insert each question and its options

Some spacing issues were fixed manually in the csv file, so the DataFrame will be loadaded from the csv file

In [7]:
processed_exam_df_filtered = pd.read_csv("../data/clean/clean_fir_2022.csv")
processed_exam_df_filtered.head()

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
0,19. ¿Cuál es la respuesta correcta respecto al...,1. Es una enterobacteria.,2. Es un bacilo gramnegativo no fermentador.,3. Es una espiroqueta.,4. Es una bacteria intracelular.,1,2022
1,20. ¿Cuál es la respuesta INCORRECTA respecto ...,1. Pertenece a la familia Paramyxoviridae.,2. El único hospedador es el ser humano.,3. Puede producir epidídimo-orquitis.,4. Es un virus ARN monocatenario con simetría ...,4,2022
2,21. ¿Cuál de las siguientes respuestas es corr...,1. Es un virus ARN bicatenario de simetría hel...,2. Reconoce el receptor humano ACE2 (enzima co...,3. El gen estructural S es idéntico al del vir...,4. Pertenece a la familia Orthomyxoviridae.,2,2022
3,22. ¿Por qué es difícil el aislamiento microbi...,1. Solo crece en anaerobiosis tras una incubac...,2. Necesita medios de cultivo enriquecidos con...,3. Necesita medios de cultivo enriquecidos con...,4. Necesita sistemas de cultivo celulares y un...,2,2022
4,23. ¿Cuál de las siguientes afirmaciones es IN...,1. Son pequeños cocobacilos o bacilos gramnega...,2. Bartonella bacilliformis es responsable de ...,3. Bartonella quintana se transmite de persona...,4. Bartonella henselae se transmite a través d...,4,2022


In [8]:
processed_exam_df_filtered.tail(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
98,196. La reacción en el centro germinal es una ...,"1. La expansión clonal, la proliferación, el c...","2. La hipermutación somática, la edición del T...","3. La expansión clonal, el cambio de isotipo, ...","4. La hipermutación somática, el reordenamient...",3,2022
99,197. La activación de las células B para produ...,1. Como contacto directo con una célula T coop...,2. Como contacto directo con una célula T coop...,3. Como contacto directo con una célula T coop...,4. Como contacto directo con una célula T coop...,3,2022
100,198. ¿Cuál de las siguientes moléculas partici...,1. CD3.,2. CD4 y CD8.,3. CD5.,4. CD28.,4,2022
101,199. Indique cuál de las siguientes afirmacion...,1. Las IDs que afectan a los mecanismos de fag...,2. Las IDs de proteínas del sistema del comple...,3. La severidad de las IDs causadas por linfoc...,4. Las IDs primarias son congénitas y las secu...,2,2022
102,202. ¿Cuál de los siguientes microorganismos s...,1. Streptobacillus moniliformis.,2. Bartonella bacilliformis.,3. Rickettsia typhi.,4. Coxiella burnetti.,1,2022
103,204. ¿Cuál de los siguientes antivíricos está ...,1. Foscarnet.,2. Letermovir.,3. Cidofovir.,4. Valganciclovir.,2,2022
104,205. Señale la opción correc ta en relación a ...,1. El factor activador de plaquetas pertenece ...,2. Son lípidos insaponificables si tienen carg...,"3. El fosfatidilinositol, que se encuentra lib...",4. La cardiolipina es el principal componente ...,1,2022
105,206. ¿Cuál de los siguientes antibióticos es i...,1. Ceftriaxona.,2. Levofloxacino.,3. Linezolid.,4. Daptomicina.,4,2022
106,208. Los llamados inhibidores mixtos son un ti...,1. Se denominan así en referencia a que pueden...,"2. Afectan a la Km aparente, pero no a la velo...","3. En la práctica, sólo se observan en enzimas...",4. Pueden provocar un aumento o disminución de...,4,2022
107,"210. En un método analítico, la concordancia e...",1. La selectividad.,2. La precisión.,3. La exactitud.,4. La sensibilidad.,3,2022


In [10]:
with sqlite3.connect(db_path) as bir_warehouse:
    cur = bir_warehouse.cursor()
    for question in processed_exam_df_filtered.itertuples():
        cur.execute(query_question, (str(question[7]), exam_acronym, question[1]))
        bir_warehouse.commit()
        for n, option in enumerate(question[2:6]):
            cur.execute(query_options, (question[1], str(year), n+1, option, n+1 == question[6]))
            bir_warehouse.commit()